In [291]:
%matplotlib inline

import sys
import matplotlib.pyplot as plt
import matplotlib.image as image
import numpy as np
import seaborn as sns
import pandas as pd
from PIL import Image
from scipy import stats
from scipy.spatial import distance
import os
import math
from sklearn.metrics.pairwise import cosine_similarity

### using cosine similarity to judge style similarity and euclidean distance to measure output similarity, maybe give people the option to weigh the importance of style and output similarities. for example, are you looking for a young player in the same mold that can be developed and produce better numbers in the future or a ready made start with similar output already

In [416]:
def create_ovrdf(df,squaddf):
    df = df[['player','position','squad','age','minutes','dribble_tackles_pct','fouls','pressures','passes_completed_long','passes_into_final_third','clearances','aerials_won_pct','aerials_won','interceptions','tackles','passes_completed','passes','miscontrols','dispossessed','passes_pct','progressive_passes','carry_progressive_distance','xa_per90','dribbles_completed','fouled','pressure_regains','touches_att_pen_area','npxg_per90','npxg_per_shot','passes_into_penalty_area','shots_total','touches','shots_total_per90']]
    fivehundredmins_filter = df['minutes'] >= 500
    df = df.loc[fivehundredmins_filter,:]
    opoku = df[df['player'] == 'Nicholas Opoku'].index
    df = df.drop(opoku)
    
    #we need to edit for per90
    for i, row in df.iterrows():
        df.at[i,'fouls'] = df.at[i,'fouls']/df.at[i,'minutes']*90
        df.at[i,'passes_completed_long'] = df.at[i,'passes_completed_long']/df.at[i,'minutes']*90
        df.at[i,'clearances'] = df.at[i,'clearances']/df.at[i,'minutes']*90
        df.at[i,'interceptions'] = df.at[i,'interceptions']/df.at[i,'minutes']*90
        df.at[i,'tackles'] = df.at[i,'tackles']/df.at[i,'minutes']*90
        df.at[i,'aerials_won'] = df.at[i,'aerials_won']/df.at[i,'minutes']*90
        df.at[i,'passes_into_final_third'] = df.at[i,'passes_into_final_third']/df.at[i,'minutes']*90
        df.at[i,'pressures'] = df.at[i,'pressures']/df.at[i,'minutes']*90
        df.at[i,'pressure_regains'] = df.at[i,'pressure_regains']/df.at[i,'minutes']*90
        df.at[i,'aerials_won'] = df.at[i,'aerials_won']/df.at[i,'minutes']*90
        df.at[i,'dribbles_completed'] = df.at[i,'dribbles_completed']/df.at[i,'minutes']*90
        df.at[i,'carry_progressive_distance'] = df.at[i,'carry_progressive_distance']/df.at[i,'minutes']*90
        df.at[i,'fouled'] = df.at[i,'fouled']/df.at[i,'minutes']*90
        df.at[i,'touches_att_pen_area'] = df.at[i,'touches_att_pen_area']/df.at[i,'minutes']*90
        df.at[i,'fouled'] = df.at[i,'fouled']/df.at[i,'minutes']*90
        df.at[i,'passes_into_penalty_area'] = df.at[i,'passes_into_penalty_area']/df.at[i,'minutes']*90
        df.at[i,'shots_total'] = df.at[i,'shots_total']/df.at[i,'minutes']*90

        #progressive pass percentage:
        df.at[i,'progressive_passes'] = df.at[i,'progressive_passes']/df.at[i,'minutes']*90
        df.at[i,'progressive_pass_percentage'] = df.at[i,'progressive_passes']/df.at[i,'passes']*100

        #shot touch%
        df.at[i,'shot_touch_perc'] = df.at[i,'shots_total']/df.at[i,'touches']*100

        #turnovers:

        df.at[i,'passes'] = df.at[i,'passes']/df.at[i,'minutes']*90
        df.at[i,'passes_completed'] = df.at[i,'passes_completed']/df.at[i,'minutes']*90
        df.at[i,'miscontrols'] = df.at[i,'miscontrols']/df.at[i,'minutes']*90
        df.at[i,'dispossessed'] = df.at[i,'dispossessed']/df.at[i,'minutes']*90

        df.at[i,'turnovers'] = df.at[i,'passes']-df.at[i,'passes_completed']+df.at[i,'miscontrols']+df.at[i,'dispossessed']

    #possesion adjusted stats
    for index, row in df.iterrows():
        df.at[index,'tackles'] = df.at[index,'tackles'] * 2/(1 + math.exp(-0.1*(squaddf.loc[squaddf['squad']==df.at[index,'squad'],'possession'].tolist()[0]-50)))
        df.at[index,'interceptions'] = df.at[index,'interceptions'] * 2/(1 + math.exp(-0.1*(squaddf.loc[squaddf['squad']==df.at[index,'squad'],'possession'].tolist()[0]-50)))
        
    df = df.rename(columns={'passes_completed_long' : 'passes_completed_long_per90',
                               'pressures': 'pressures_per90',
                               'clearances': 'clearances_per90',
                                'interceptions': 'interceptions_per90',
                                'tackles': 'tackles_per90',
                                'fouls': 'fouls_per90',
                                'aerials_won': 'aerials_won_per90',
                                'passes_into_final_third' : 'passes_into_final_third_per90',
                                'pressure_regains': 'pressure_regains_per90',
                                'dribbles_completed': 'dribbles_completed_per90',
                                'progressive_passes': 'progressive_passes_per90',
                                'carry_progressive_distance' : 'carry_progressive_distance_per90',
                                'fouled': 'fouled_per90',
                                'touches_att_pen_area': 'touches_att_pen_area_per90',
                                'turnovers': 'turnovers_per90',
                                'passes_into_penalty_area': 'passes_into_penalty_area_per90',
                               })
    
    df['index'] = list(range(len(df.index)))
    df = df.set_index('index')
    
    return df        

In [ ]:
def create_ovrdf(df,squaddf):
    df = df[['player','position','squad','age','minutes','dribble_tackles_pct','fouls','pressures','passes_completed_long','passes_into_final_third','clearances','aerials_won_pct','aerials_won','interceptions','tackles','passes_completed','passes','miscontrols','dispossessed','passes_pct','progressive_passes','carry_progressive_distance','xa_per90','dribbles_completed','fouled','pressure_regains','touches_att_pen_area','npxg_per90','npxg_per_shot','passes_into_penalty_area','shots_total','touches','shots_total_per90']]
    fivehundredmins_filter = df['minutes'] >= 500
    df = df.loc[fivehundredmins_filter,:]
    opoku = df[df['player'] == 'Nicholas Opoku'].index
    df = df.drop(opoku)
    
    #we need to edit for per90
    for i, row in df.iterrows():
        df.at[i,'fouls'] = df.at[i,'fouls']/df.at[i,'minutes']*90
        df.at[i,'passes_completed_long'] = df.at[i,'passes_completed_long']/df.at[i,'minutes']*90
        df.at[i,'clearances'] = df.at[i,'clearances']/df.at[i,'minutes']*90
        df.at[i,'interceptions'] = df.at[i,'interceptions']/df.at[i,'minutes']*90
        df.at[i,'tackles'] = df.at[i,'tackles']/df.at[i,'minutes']*90
        df.at[i,'aerials_won'] = df.at[i,'aerials_won']/df.at[i,'minutes']*90
        df.at[i,'passes_into_final_third'] = df.at[i,'passes_into_final_third']/df.at[i,'minutes']*90
        df.at[i,'pressures'] = df.at[i,'pressures']/df.at[i,'minutes']*90
        df.at[i,'pressure_regains'] = df.at[i,'pressure_regains']/df.at[i,'minutes']*90
        df.at[i,'aerials_won'] = df.at[i,'aerials_won']/df.at[i,'minutes']*90
        df.at[i,'dribbles_completed'] = df.at[i,'dribbles_completed']/df.at[i,'minutes']*90
        df.at[i,'carry_progressive_distance'] = df.at[i,'carry_progressive_distance']/df.at[i,'minutes']*90
        df.at[i,'fouled'] = df.at[i,'fouled']/df.at[i,'minutes']*90
        df.at[i,'touches_att_pen_area'] = df.at[i,'touches_att_pen_area']/df.at[i,'minutes']*90
        df.at[i,'fouled'] = df.at[i,'fouled']/df.at[i,'minutes']*90
        df.at[i,'passes_into_penalty_area'] = df.at[i,'passes_into_penalty_area']/df.at[i,'minutes']*90
        df.at[i,'shots_total'] = df.at[i,'shots_total']/df.at[i,'minutes']*90

        #progressive pass percentage:
        df.at[i,'progressive_passes'] = df.at[i,'progressive_passes']/df.at[i,'minutes']*90
        df.at[i,'progressive_pass_percentage'] = df.at[i,'progressive_passes']/df.at[i,'passes']*100

        #shot touch%
        df.at[i,'shot_touch_perc'] = df.at[i,'shots_total']/df.at[i,'touches']*100

        #turnovers:

        df.at[i,'passes'] = df.at[i,'passes']/df.at[i,'minutes']*90
        df.at[i,'passes_completed'] = df.at[i,'passes_completed']/df.at[i,'minutes']*90
        df.at[i,'miscontrols'] = df.at[i,'miscontrols']/df.at[i,'minutes']*90
        df.at[i,'dispossessed'] = df.at[i,'dispossessed']/df.at[i,'minutes']*90

        df.at[i,'turnovers'] = df.at[i,'passes']-df.at[i,'passes_completed']+df.at[i,'miscontrols']+df.at[i,'dispossessed']

    #possesion adjusted stats
    for index, row in df.iterrows():
        df.at[index,'tackles'] = df.at[index,'tackles'] * 2/(1 + math.exp(-0.1*(squaddf.loc[squaddf['squad']==df.at[index,'squad'],'possession'].tolist()[0]-50)))
        df.at[index,'interceptions'] = df.at[index,'interceptions'] * 2/(1 + math.exp(-0.1*(squaddf.loc[squaddf['squad']==df.at[index,'squad'],'possession'].tolist()[0]-50)))
        
    df = df.rename(columns={'passes_completed_long' : 'passes_completed_long_per90',
                               'pressures': 'pressures_per90',
                               'clearances': 'clearances_per90',
                                'interceptions': 'interceptions_per90',
                                'tackles': 'tackles_per90',
                                'fouls': 'fouls_per90',
                                'aerials_won': 'aerials_won_per90',
                                'passes_into_final_third' : 'passes_into_final_third_per90',
                                'pressure_regains': 'pressure_regains_per90',
                                'dribbles_completed': 'dribbles_completed_per90',
                                'progressive_passes': 'progressive_passes_per90',
                                'carry_progressive_distance' : 'carry_progressive_distance_per90',
                                'fouled': 'fouled_per90',
                                'touches_att_pen_area': 'touches_att_pen_area_per90',
                                'turnovers': 'turnovers_per90',
                                'passes_into_penalty_area': 'passes_into_penalty_area_per90',
                               })
    
    df['index'] = list(range(len(df.index)))
    df = df.set_index('index')
    
    return df        

In [362]:
def create_ovr_graphdf(df):
    graphdf = df[['dribble_tackles_pct','fouls_per90','pressures_per90','passes_completed_long_per90','passes_into_final_third_per90','clearances_per90','aerials_won_pct','aerials_won_per90','interceptions_per90','tackles_per90','passes_pct','progressive_pass_percentage','progressive_passes_per90','carry_progressive_distance_per90','xa_per90','dribbles_completed_per90','fouled_per90','pressure_regains_per90','touches_att_pen_area_per90','npxg_per90','npxg_per_shot','passes_into_penalty_area_per90','shots_total_per90','shot_touch_perc','turnovers_per90']]
    return graphdf

In [298]:
def leagueName(leagueAbbr):
    if leagueAbbr == 'B5':
        return "Top 5 European Leagues"
    elif leagueAbbr == 'PL':
        return "Premier League"
    elif leagueAbbr == 'Bu':
        return "Bundesliga"
    elif leagueAbbr == 'SA':
        return "Serie A"
    elif leagueAbbr == 'LL':
        return "La Liga"
    elif leagueAbbr == 'L1':
        return "Ligue 1"

In [413]:
def findSimilar(playerName,playerLeague,scoutLeague,statsProfile,styleFactor,outputFactor,ageLow,ageHigh,numPlayers):
    
    if styleFactor+outputFactor != 1:
        return('Input style and output factors that add up to 1')
    
    ###############################
    # create player data
    df = import_player_data(playerLeague,1920)
    squaddf = import_squad_data(playerLeague,1920)
    
    df = create_ovrdf(df,squaddf)
    allplayers = create_ovr_graphdf(df)

    stats = []
    for i in allplayers.columns:
        stats.append(i)

    #for i, row in df.iterrows():
    for i in stats:
        allplayers.loc[:,i] = allplayers.loc[:,i].rank(pct=True) # will be in decimal form

    player = allplayers.iloc[[df[df['player']==playerName].index.values[0]]]
    
    ################################
    # create scout data
    df = import_player_data(scoutLeague,1920)
    squaddf = import_squad_data(scoutLeague,1920)

    # age range
    ageLowfilter = df.loc[:,'age'] >= ageLow
    ageHighfilter = df.loc[:,'age'] <= ageHigh
    df = df.loc[ageLowfilter & ageHighfilter,:]
    
    df = create_ovrdf(df,squaddf)
    scoutedPlayers = create_ovr_graphdf(df)

    #for i, row in df.iterrows():
    for i in stats:
        scoutedPlayers.loc[:,i] = scoutedPlayers.loc[:,i].rank(pct=True) # will be in decimal form
    
    eucDist = []
    for i, row in scoutedPlayers.iterrows():
        eucDist.append(distance.euclidean(player, row))
        
    maxEucDist = max(eucDist)
    for i, row in scoutedPlayers.iterrows():
        eucDist[i] = 1 - eucDist[i]/maxEucDist
    eucDist = np.array(eucDist).reshape(len(eucDist),1)
    
    cos_dist = cosine_similarity(player,scoutedPlayers)
    cos_dist = np.transpose(cos_dist)

    scoutedPlayers.loc[:,'euclidian_similarity'] = eucDist
    scoutedPlayers.loc[:,'cosine_similarity'] = cos_dist
    euc_dist_sorted = pd.DataFrame(eucDist).sort_values(by=[0], ascending=False)
    cos_dist_sorted = pd.DataFrame(cos_dist).sort_values(by=[0], ascending=False)
    
    roleSuitability = outputFactor*eucDist + styleFactor*cos_dist

    scoutedPlayers.loc[:,'role_suitability'] = roleSuitability
    
    role_suitability_sorted = pd.DataFrame(roleSuitability).sort_values(by=[0], ascending=False)

    n = 0
    print('Transfer Shortlist and Suitability With Given Style and Output Desires:'.format(playerName,leagueName(scoutLeague)))
    for i, row in role_suitability_sorted.iterrows():
        if df.iloc[i]['player'] == playerName:
            continue
        print('{}:'.format(df.iloc[i]['player']),'{}%'.format(round(row[0]*100,5)))
        n+=1
        if n == numPlayers:
            break
            
    n = 0
    print('\nPlayers With Similar Output Level to {} in the {}:'.format(playerName,leagueName(scoutLeague)))
    for i, row in euc_dist_sorted.iterrows():
        if df.iloc[i]['player'] == playerName:
            continue
        print('{}:'.format(df.iloc[i]['player']),'{}%'.format(round(row[0]*100,5)))
        n+=1
        if n == numPlayers:
            break
    n = 0  
    print('\nPlayers Most Stylistically Similar to {} in the {}:'.format(playerName,leagueName(scoutLeague)))
    for i, row in cos_dist_sorted.iterrows():
        if df.iloc[i]['player'] == playerName:
            continue
        print('{}:'.format(df.iloc[i]['player']),'{}%'.format(round(row[0]*100,5)))
        n+=1
        if n == numPlayers:
            break

In [427]:
findSimilar('Ben Chilwell','PL','B5','n/a',0.75,0.25,20,28,20)

Transfer Shortlist and Suitability With Given Style and Output Desires:
Sergio Reguilón: 92.09497%
Maximilian Mittelstädt: 91.31137%
Issiaga Sylla: 91.19333%
Emerson: 90.8843%
Pervis Estupiñán: 90.48761%
Ricardo Pereira: 90.3851%
Leo Dubois: 90.33108%
Ola Aina: 90.28946%
Hamari Traoré: 90.25367%
Theo Hernández: 90.01363%
Ainsley Maitland-Niles: 89.98075%
Stefan Lainer: 89.76337%
Ander Capa: 89.65626%
Lucas Digne: 89.50968%
Alfonso Pedraza: 89.43622%
Bouna Sarr: 89.218%
Marcel Halstenberg: 89.21146%
Alex Sandro: 89.20745%
Mikel Merino: 89.12904%
Hugo Mallo: 88.71964%

Players With Similar Output Level to Ben Chilwell in the Top 5 European Leagues:
Sergio Reguilón: 75.66137%
Maximilian Mittelstädt: 74.25404%
Issiaga Sylla: 73.76674%
Emerson: 73.1046%
Pervis Estupiñán: 72.31449%
Leo Dubois: 72.00182%
Hamari Traoré: 71.78193%
Ola Aina: 71.69693%
Ainsley Maitland-Niles: 71.19951%
Ricardo Pereira: 70.85849%
Stefan Lainer: 70.73011%
Lucas Digne: 70.1716%
Theo Hernández: 70.16326%
Alfonso Pedr

C:\Users\jpb31\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\jpb31\anaconda3\lib\site-packages\pandas\core\indexing.py:1047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [190]:
df = import_squad_data('PL',1920)
#df = import_player_data('PL',1920)
#for i in df.columns:
#    print(i)
    
allteams = df[['squad','players_used','possession','games','games_starts','minutes','goals']]
ars = df.loc[df['squad']=='Arsenal',:][['players_used','possession','games','games_starts','minutes','goals']]
print(ars)
comp = pd.DataFrame.transpose(pd.DataFrame(cosine_similarity(ars,allteams[['players_used','possession','games','games_starts','minutes','goals']])))
#print(df['squad'])
allteams.join(comp)
comp.sort_values(by=[0], ascending=False)
for i, row in comp.iterrows():
    print('{}:'.format(allteams.iloc[i]['squad']),'{}%'.format(round(row[0]*100,5)))

   players_used  possession  games  games_starts  minutes  goals
0          29.0        54.0   38.0         418.0   3420.0   56.0
Arsenal: 100.0%
Aston Villa: 99.99849%
Bournemouth: 99.99818%
Brighton: 99.99806%
Burnley: 99.99818%
Chelsea: 99.99908%
Crystal Palace: 99.99649%
Everton: 99.99897%
Leicester City: 99.9995%
Liverpool: 99.99646%
Manchester City: 99.99105%
Manchester Utd: 99.99964%
Newcastle Utd: 99.99763%
Norwich City: 99.99586%
Sheffield Utd: 99.99778%
Southampton: 99.99973%
Tottenham: 99.99999%
Watford: 99.99755%
West Ham: 99.99937%
Wolves: 99.99939%


In [191]:
df = import_player_data('PL',1920)

df = create_stdf(df)
allplayers = create_st_graphdf(df)

stats = []
for i in values.columns:
    stats.append(i)
    
for i in stats:
    allplayers[i] = allplayers[i].rank(pct=True) # will be in decimal form

player = allplayers.iloc[[df[df['player']=='Troy Deeney'].index.values[0]]]

cos_dist = np.transpose(cosine_similarity(player,allplayers))

allplayers['cosine_similarity'] = cos_dist

cos_dist_sorted = pd.DataFrame(cos_dist).sort_values(by=[0], ascending=False)

for i, row in cos_dist_sorted.iterrows():
    print('{}:'.format(df.iloc[i]['player']),'{}%'.format(round(row[0]*100,5)))




Troy Deeney: 100.0%
Shane Long: 94.50212%
Sébastien Haller: 93.85986%
Oliver McBurnie: 89.89732%
Glenn Murray: 88.25436%
Joelinton: 88.19156%
Andy Carroll: 86.13288%
David McGoldrick: 85.31048%
Andre Gray: 85.04951%
Divock Origi: 83.49159%
Wesley Moraes: 82.08336%
Richarlison: 81.98468%
Callum Wilson: 81.90497%
Martinelli: 78.71162%
Billy Sharp: 77.99666%
Christian Benteke: 77.85599%
Sadio Mané: 77.21172%
Michael Obafemi: 76.30201%
Che Adams: 76.25676%
Daniel James: 75.54695%
Michail Antonio: 74.78552%
Alexandre Lacazette: 73.54921%
Willian: 73.37459%
Dominic Calvert-Lewin: 73.30303%
Keinan Davis: 72.7669%
Raúl Jiménez: 72.40125%
Olivier Giroud: 72.33285%
Mbwana Samatta: 72.05893%
Christian Pulisic: 71.85631%
Jay Rodriguez: 71.79435%
Danny Ings: 71.71382%
Jarrod Bowen: 71.62084%
Pierre-Emerick Aubameyang: 71.42972%
Matěj Vydra: 71.39022%
Nicolas Pépé: 70.86648%
Eddie Nketiah: 70.42299%
Lys Mousset: 69.34105%
Roberto Firmino: 68.84566%
Ashley Barnes: 68.00346%
Chris Wood: 67.43349%
Call

C:\Users\jpb31\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\jpb31\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [278]:
df = import_player_data('L1',1920)
squaddf = import_squad_data('L1',1920)

df = create_ovrdf(df,squaddf)
allplayers = create_ovr_graphdf(df)

stats = []
for i in allplayers.columns:
    stats.append(i)

#for i, row in df.iterrows():
for i in stats:
    allplayers[i] = allplayers[i].rank(pct=True) # will be in decimal form

player = allplayers.iloc[[df[df['player']=='Memphis Depay'].index.values[0]]]

#cos_dist = cosine_similarity(player,allplayers)
#cos_dist = np.transpose(cos_dist)

#df = df.drop(267, axis=0)
opoku = df[df['player'] == 'Nicholas Opoku'].index
df = df.drop(opoku)
allplayers.isnull().sum()
df[df['progressive_pass_percentage'].isnull()]
df['index'] = list(range(len(df.index)))
df = df.set_index('index')
df.loc[df['player']=='Victor Osimhen']


C:\Users\jpb31\anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\jpb31\anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\jpb31\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,player,position,squad,age,minutes,dribble_tackles_pct,fouls_per90,pressures_per90,passes_completed_long_per90,passes_into_final_third_per90,...,pressure_regains_per90,touches_att_pen_area_per90,npxg_per90,npxg_per_shot,passes_into_penalty_area_per90,shots_total,touches,shots_total_per90,shot_touch_perc,turnovers_per90
index,,,,,,,,,,,,,,,,,,,,,
267,Victor Osimhen,FW,Lille,20,2287.0,12.5,2.243113,17.275907,0.944469,0.432882,...,5.35199,6.217753,0.5,0.15,0.196764,3.226935,704.0,3.23,0.458371,10.428509
